In [ ]:
# dowload the dataset from google drive 
! gdown --id 1CX3x0YmRmDQivhjfgB0kVzfXzVql7Axj

# unzip the dataset 
!unzip /content/stage1_train.zip

# dowload the basic functions
! gdown --id 1ZFtLIVLDyEX_AoK22peWdszOanvKxGyi
! gdown --id 1YF2GqFbLPdIsPCODZ4fA3mWaprPuNjuR

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import os
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.pytorch import ToTensorV2
from skimage import io, transform
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import data_preprocess as dp

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
## Access to the google drive
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
class Nuclie_data(Dataset):
        def __init__(self,path):
            self.path = path
            self.folders = os.listdir(path)
            self.transforms = get_transforms(0.5, 0.5)
        
        def __len__(self):
            return len(self.folders)
              
        
        def __getitem__(self,idx):
            image_folder = os.path.join(self.path,self.folders[idx],'images/')
            mask_folder = os.path.join(self.path,self.folders[idx],'masks/')
            image_path = os.path.join(image_folder,os.listdir(image_folder)[0])
            
            #convert the RGBA into RGB
            img = io.imread(image_path)[:,:,:3].astype('float32')
            img = transform.resize(img,(128,128))

            masks=[]
            for mskName in os.listdir(mask_folder):
                vesMask = (cv2.imread(mask_folder+'/'+mskName, 0) > 0).astype(np.uint8)  # Read vesse instance mask
                vesMask=cv2.resize(vesMask,[128,128],cv2.INTER_NEAREST)
                masks.append(vesMask)
            
            #mask = get_mask(mask_folder, 128, 128 ).astype('float32')
            boxes = get_bounding_box(mask_folder, 128, 128 )
          
            # convert everything into a torch.Tensor
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            masks = torch.as_tensor(masks, dtype=torch.uint8)
            #img = torch.as_tensor(img, dtype=torch.float32)
            image_id = torch.tensor([idx])
            
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            
            #augmentation
            augmented = self.transforms(image=img)
            img = augmented['image']
            #mask = augmented['mask']
            num_objs = len(boxes)
            
            # suppose all instances are not crow
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
            
            target = {}
            target["boxes"] = boxes 

            #there is only one class 
            target["labels"] =  torch.ones((num_objs,), dtype=torch.int64)   # there is only one class
            target["masks"] = masks
            target["image_id"] = image_id
            target["area"] = area
            target["iscrowd"] = iscrowd
            
            return (img,target)

In [ ]:
##get the bounding box for each individual mask 
def get_bounding_box(mask_folder,IMG_HEIGHT, IMG_WIDTH):
    boxes = []
    #masks = []
    for mask_ in os.listdir(mask_folder):
            mask_ = io.imread(os.path.join(mask_folder,mask_)).astype('float32')
            mask_ = transform.resize(mask_, (IMG_HEIGHT, IMG_WIDTH))
            ###get the bounding box
            pos = np.where(mask_)
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
            #masks.append(mask_)
              
    return boxes

In [ ]:
def get_transforms(mean, std):
            list_transforms = []
            
            list_transforms.extend(
                    [
                HorizontalFlip(p=0), # only horizontal flip as of now
                    ])
            list_transforms.extend(
                    [
            Normalize(mean=mean, std=std, p=1),
            ToTensorV2(),
                    ])
            list_trfms = Compose(list_transforms)
            return list_trfms

In [ ]:
#loading the datad
data_dir = '/content/stage1_train/'
data = Nuclie_data(data_dir)

In [ ]:
#data size
print(data.__len__())

670


## Evaluate Semantic Segmentation

In [ ]:
import torchvision.models.segmentation
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [ ]:
def get_model_semantic(num_classes):
    #load the pre-trained model 
    model=torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True) 
    in_features = model.roi_heads.box_predictor.cls_score.in_features 
    model.roi_heads.box_predictor=FastRCNNPredictor(in_features,num_classes)

    return model

In [ ]:
## get trained models
model_path = "/content/MyDrive/MyDrive/trained_model/Combined_masks/"
model_list = []
# Iterate directory
for file in os.listdir(model_path):
    # check only text files
    if file.endswith('.torch'):
        model_list.append(file)
model_list

['1000_augmentation_batch2.torch',
 '200_augmentation_batch4.torch',
 '400_augmentation_batch4.torch',
 '600_augmentation_batch4.torch',
 '800_augmentation_batch4.torch',
 '1000_augmentation_batch4.torch',
 '200_augmentation_batch6.torch',
 '400_augmentation_batch6.torch',
 '600_augmentation_batch6.torch',
 '800_augmentation_batch6.torch',
 '1000_augmentation_batch6.torch',
 '200_augmentation_batch8.torch',
 '400_augmentation_batch8.torch',
 '600_augmentation_batch8.torch',
 '800_augmentation_batch8.torch',
 '1000_augmentation_batch8.torch',
 '200_augmentation_batch10.torch',
 '400_augmentation_batch10.torch',
 '600_augmentation_batch10.torch',
 '800_augmentation_batch10.torch',
 '1000_augmentation_batch10.torch',
 '1000_augmentation_batch1.torch']

In [ ]:
for trained_model in model_list:

    model = get_model_semantic(2)

    print("Evaluating the model:", trained_model)
    model.load_state_dict(torch.load(os.path.join(model_path, trained_model)))
    model.eval()
    model.to(device)# move model to the right device

    evaluator = dp.evaluate_IoU(data, model, range(581, 670), device)
    print(" ")
    print(" ")


## Instance segamentation 

In [ ]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
model_path = "/content/MyDrive/MyDrive/trained_model/individual_mask/"
model_list = []
# Iterate directory
for file in os.listdir(model_path):
    # check only text files
    if file.endswith('.torch'):
        model_list.append(file)
print(model_list)

['500_individal_batch4.torch', '1000_individal_batch4.torch', '1500_individal_batch4.torch', '2000_individal_batch4.torch', '500_individal_batch6.torch', '1000_individal_batch6.torch', '1500_individal_batch6.torch', '2000_individal_batch6.torch', '500_individal_batch8.torch', '1000_individal_batch8.torch', '1500_individal_batch8.torch', '2000_individal_batch8.torch', '500_individal_batch10.torch', '1000_individal_batch10.torch', '1500_individal_batch10.torch', '2000_individal_batch10.torch']


In [ ]:
model_list

['500_individal_batch4.torch',
 '1000_individal_batch4.torch',
 '1500_individal_batch4.torch',
 '2000_individal_batch4.torch',
 '500_individal_batch6.torch',
 '1000_individal_batch6.torch',
 '1500_individal_batch6.torch',
 '2000_individal_batch6.torch',
 '500_individal_batch8.torch',
 '1000_individal_batch8.torch',
 '1500_individal_batch8.torch',
 '2000_individal_batch8.torch',
 '500_individal_batch10.torch',
 '1000_individal_batch10.torch',
 '1500_individal_batch10.torch',
 '2000_individal_batch10.torch']

In [ ]:
for trained_model in model_list:

    model = get_model_instance_segmentation(2)

    print("Evaluating the model:", trained_model)
    model.load_state_dict(torch.load(os.path.join(model_path, trained_model)))
    model.eval()
    model.to(device)# move model to the right device

    evaluator = dp.evaluate_IoU(data, model, range(581, 670), device)
    print(" ")
    print(" ")